In [1]:
from sampo.generator import SimpleSynthetic
from sampo.generator.types import SyntheticGraphType

# 1. Graph generation

In [2]:
# SimpleSynthetic object used for the simple work graph structure generation

r_seed = 231
ss = SimpleSynthetic(r_seed)

In [3]:
# simple graph
# should generate general (average) type of graph with 10 clusters from 100 to 200 vertices each

simple_wg = ss.work_graph(mode=SyntheticGraphType.General,
                          cluster_counts=10,
                          bottom_border=100,
                          top_border=200)

In [4]:
# complex graph
# should generate general (average) type of graph with 300 unique works, 100 resources and 2000 vertices

advanced_wg = ss.advanced_work_graph(works_count_top_border=2000,
                                     uniq_works=300,
                                     uniq_resources=100)

# 2. Contractor generation

In [5]:
from uuid import uuid4
from sampo.schemas.resources import Worker
from sampo.schemas.contractor import Contractor

### Manual generation
To create contractor, you should provide minimal info: unique id, contractor name, and supplied workers (simple renewable resources).

In [6]:
contractors = [
    Contractor(id=str(uuid4()),
               name="OOO Berezka",
               workers={'worker' : Worker(id='0', name='worker', count=100)})
]

### Generation from graph

In [7]:
from sampo.generator import get_contractor_by_wg

contractors = [get_contractor_by_wg(simple_wg)]

# 3. Scheduling

### Scheduler construction
Before scheduling you should specify scheduling algorithm used for transforming input data to the final schedule.
At this time SAMPO contains heuristic algorithms, such as HEFTAddEnd, HEFTAddBetween and Topological scheduler, and the Genetic algorithm.
While creation, you can specify the hyperparameters to fit the algorithm.

In [8]:
from sampo.scheduler.heft.base import HEFTScheduler

# here we can just create simple heuristic scheduler
scheduler = HEFTScheduler()

In [9]:
from sampo.scheduler.genetic.base import GeneticScheduler

# or more complex genetic scheduler
scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3)

Can't find native module; switching to default


### Scheduling process
SAMPO provides a simple interface to all its features.
It called SchedulingPipeline.
Using it you only should pass all the scheduling arguments, it remembers them, and you can produce schedules in many ways.

In [10]:
from sampo.pipeline import SchedulingPipeline

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Genetic optimizing took 0.0 ms
Toolbox initialization & first population took 387.9549503326416 ms
First population evaluation took 390.5296325683594 ms
-- Generation 0, population=20, best time=994.0 --
-- Generation 1, population=15, best time=984.0 --
-- Generation 2, population=10, best time=984.0 --
-- Generation 3, population=10, best time=984.0 --
-- Generation 4, population=4, best time=984.0 --
-- Generation 5, population=4, best time=984.0 --
-- Generation 6, population=6, best time=984.0 --
-- Generation 7, population=6, best time=984.0 --
-- Generation 8, population=18, best time=984.0 --
-- Generation 9, population=26, best time=984.0 --
-- Generation 10, population=18, best time=984.0 --
-- Generation 11, population=9, best time=984.0 --
-- Generation 12, population=14, best time=984.0 --
-- Generation 13, population=7, best time=984.0 --
-- Generation 14, population=2, best time=984.0 --
-- Generation 15, population=1, best time=984.0 --
-- Generation 16, population=1, b

984

### Other metrics
Genetic scheduler can do more than just optimize MakeSpan. It supports more metrics.
Metrics are represented by `FitnessFunction` class and can be passed to `GeneticScheduler` on constructing.

Here we're constructing Genetic that should optimize resources to deadline.

In [11]:
from sampo.schemas.time import Time
from sampo.scheduler.genetic.operators import DeadlineResourcesFitness

deadline = Time(2000)
# calling `prepare` method to pass explicit parameters to fitness function
fitness_constructor = DeadlineResourcesFitness.prepare(deadline)

scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Genetic optimizing took 0.0 ms
Toolbox initialization & first population took 363.64173889160156 ms
First population evaluation took 421.886682510376 ms
-- Generation 0, population=20, best time=375.0 --
-- Generation 1, population=11, best time=371.0 --
-- Generation 2, population=5, best time=371.0 --
-- Generation 3, population=8, best time=371.0 --
-- Generation 4, population=9, best time=360.0 --
-- Generation 5, population=3, best time=360.0 --
-- Generation 6, population=1, best time=360.0 --
-- Generation 7, population=1, best time=360.0 --
-- Generation 8, population=1, best time=360.0 --
-- Generation 9, population=1, best time=360.0 --
-- Generation 10, population=1, best time=360.0 --
-- Generation 11, population=1, best time=360.0 --
-- Generation 12, population=1, best time=360.0 --
-- Generation 13, population=1, best time=360.0 --
-- Generation 14, population=1, best time=360.0 --
-- Generation 15, population=1, best time=360.0 --
-- Generation 16, population=1, best ti

1155

Additionally, you can construct other metrics: deadline cost, time with resources (without deadline) fitness:

In [15]:
from sampo.scheduler.genetic.operators import DeadlineCostFitness

fitness_constructor = DeadlineCostFitness.prepare(deadline)

scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Genetic optimizing took 0.0 ms
Toolbox initialization & first population took 156.21137619018555 ms
First population evaluation took 506.76465034484863 ms
-- Generation 0, population=20, best time=3750.0 --
-- Generation 1, population=9, best time=3750.0 --
-- Generation 2, population=4, best time=3750.0 --
-- Generation 3, population=7, best time=3710.0 --
-- Generation 4, population=9, best time=3710.0 --
-- Generation 5, population=5, best time=3710.0 --
-- Generation 6, population=4, best time=3710.0 --
-- Generation 7, population=8, best time=3640.0 --
-- Generation 8, population=12, best time=3640.0 --
-- Generation 9, population=5, best time=3640.0 --
-- Generation 10, population=6, best time=3570.0 --
-- Generation 11, population=2, best time=3570.0 --
-- Generation 12, population=1, best time=3570.0 --
-- Generation 13, population=1, best time=3570.0 --
-- Generation 14, population=1, best time=3570.0 --
-- Generation 15, population=1, best time=3570.0 --
-- Generation 16, pop

1894

In [14]:
from sampo.scheduler.genetic.operators import TimeAndResourcesFitness

fitness_constructor = TimeAndResourcesFitness

scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Genetic optimizing took 0.0 ms
Toolbox initialization & first population took 140.5961513519287 ms
First population evaluation took 406.12125396728516 ms
-- Generation 0, population=20, best time=1369.0 --
-- Generation 1, population=11, best time=1369.0 --
-- Generation 2, population=7, best time=1369.0 --
-- Generation 3, population=11, best time=1369.0 --
-- Generation 4, population=6, best time=1369.0 --
-- Generation 5, population=12, best time=1369.0 --
-- Generation 6, population=6, best time=1369.0 --
-- Generation 7, population=3, best time=1369.0 --
-- Generation 8, population=4, best time=1369.0 --
-- Generation 9, population=9, best time=1369.0 --
-- Generation 10, population=13, best time=1362.0 --
-- Generation 11, population=7, best time=1355.0 --
-- Generation 12, population=2, best time=1355.0 --
-- Generation 13, population=1, best time=1355.0 --
-- Generation 14, population=1, best time=1355.0 --
-- Generation 15, population=1, best time=1355.0 --
-- Generation 16, p

994